#### 1. Library Install & Load / Basic Setting

In [1]:
!git clone https://github.com/UKPLab/sentence-transformers.git

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 11495, done.
remote: Counting objects: 100% (3737/3737), done.
remote: Compressing objects: 100% (939/939), done.
remote: Total 11495 (delta 3279), reused 2806 (delta 2798), pack-reused 7758 (from 1)
Receiving objects: 100% (11495/11495), 25.89 MiB | 26.56 MiB/s, done.
Resolving deltas: 100% (8184/8184), done.


In [2]:
!git clone https://github.com/kakaobrain/kor-nlu-datasets.git

Cloning into 'kor-nlu-datasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 16 (delta 1), reused 1 (delta 1), pack-reused 14 (from 1)
Receiving objects: 100% (16/16), 46.67 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip uninstall sentence_transformers

Found existing installation: sentence-transformers 3.3.1
Uninstalling sentence-transformers-3.3.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/sentence_transformers-3.3.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-3.3.1


In [5]:
%cd sentence-transformers

/content/sentence-transformers


In [6]:
!pip install -e .

Obtaining file:///content/sentence-transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sentence-transformers (pyproject.toml) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-3.4.0.dev0-0.editable-py3-none-any.whl size=10980 sha256=9490a569e800f129ea0e7b9838798db6cd31cd03bcb9de7a7e500291ee9065ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-kpdqcbiz/wheels/20/79/ec/b839e68c83b7f73dfbf15cf4f989275c8d4eed6b3e247eecbd
Successfully built sentence-transformers


In [7]:
import sys
import os

# 현재 워킹 디렉토리를 /content로 변경
os.chdir("/content")

# sentence-transformers 폴더의 경로를 sys.path에 추가
sys.path.append("/content/sentence-transformers")

In [8]:
import os
import csv
import math
import torch
import random
import logging
import numpy as np

from datasets import Dataset
from datetime import datetime
from torch.utils.data import DataLoader

from sentence_transformers import (
    SentenceTransformer,
    LoggingHandler,
    datasets,
    models,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction
from sentence_transformers.readers import InputExample

In [9]:
# 로거 설정
log_file = "log_output.txt"

logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[
        logging.FileHandler(log_file),  # 파일 핸들러로 로그를 파일에 기록
        logging.StreamHandler()        # 콘솔에도 출력
    ]
)

# 시드 고정
SEED = 777
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [10]:
# 데이터 경로 설정
KORNLI_PATH = '/content/kor-nlu-datasets/KorNLI'
KORSTS_PATH = '/content/kor-nlu-datasets/KorSTS'
OUTPUT_DIR = 'output'

In [11]:
final_scores = {}

In [12]:
logging.info("설정 및 라이브러리, 데이터 git 복제 완료.")

In [13]:
# KorSTS 데이터셋을 읽어와 InputExample 리스트를 반환
def load_kor_sts_samples(filename):
    samples = []
    with open(filename, 'rt', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            score = float(row['score']) / 5.0 # Normalize
            samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [14]:
def load_kor_nli_samples(filename):
    """
    KorNLI 데이터를 읽어와서, Entailment vs Contradiction를 이용해
    MultipleNegativesRankingLoss에 맞는 형태(InputExample)를 생성.
    """
    data = {}
    def add_to_samples(sent1, sent2, label):
        if sent1 not in data:
            data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
        data[sent1][label].add(sent2)

    with open(filename, 'r', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            sent1 = row['sentence1'].strip()
            sent2 = row['sentence2'].strip()
            gold_label = row['gold_label'].strip()
            if gold_label not in ['entailment','contradiction','neutral']:
                # 혹시 라벨이 이상하면 건너뛰기
                continue
            add_to_samples(sent1, sent2, gold_label)
            add_to_samples(sent2, sent1, gold_label)

    samples = []
    for sent1, labels_dict in data.items():
        # entailment/contradiction이 각각 하나 이상 있어야 hard negative 구성 가능
        if len(labels_dict['entailment'])>0 and len(labels_dict['contradiction'])>0:
            pos = random.choice(list(labels_dict['entailment']))
            neg = random.choice(list(labels_dict['contradiction']))
            samples.append(InputExample(texts=[sent1, pos, neg]))

            # sent1을 가운데 두고, pos가 앞뒤로 바뀐 또다른 예시도 추가
            # (Optional - 다양하게 hard negative샘플을 늘릴 수 있음)
            samples.append(InputExample(texts=[pos, sent1, neg]))

    return samples

In [15]:
logging.info("Data Utilize(Pre-Processing) / Sample Extraction 완료")

In [16]:
def evaluate_on_sts_test(model_path, batch_size=64):
    """
    주어진 model_path에 있는 모델을 로드하고
    KorSTS Test 세트로 EmbeddingSimilarityEvaluator 시행 후
    최종 스코어를 반환합니다.
    """
    test_file_sts = os.path.join(KORSTS_PATH, "sts-test.tsv")
    test_samples_sts = load_kor_sts_samples(test_file_sts)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
        test_samples_sts,
        batch_size=batch_size,
        name='sts-test'
    )

    model = SentenceTransformer(model_path)
    score = test_evaluator(model)
    return score

In [18]:
import zipfile
import os

# 1. 압축 해제
zip_file_path = "/content/output.zip"
extract_path = "/content/output"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"압축 해제 완료: {extract_path}")

# 2. 원본 zip 파일 삭제
os.remove(zip_file_path)
print(f"{zip_file_path} 삭제 완료.")

압축 해제 완료: /content/output
/content/output.zip 삭제 완료.


In [22]:
# 3. 모델 경로 설정 (01-17 경로로 설정)
nli_model_path = "output/nli-only-klue-roberta-base-01-17"

In [23]:
##################################################
# 실험 4: NLI -> STS                             #
##################################################
def train_nli_then_sts(
    nli_model_path,  # 실험 2에서 학습 완료된 모델 경로
    max_seq_length=128,
    batch_size=16,
    num_epochs=4,
    output_dir=OUTPUT_DIR
):
    """
    4) NLI -> STS 실험
       이미 NLI로 학습된 모델(실험2 결과)을 불러와 STS 학습 진행
    """

    model_save_path = os.path.join(
        output_dir,
        f"nli-to-sts-{datetime.now().strftime('%Y-%m-%d')}"
    )
    logging.info(f"[NLI->STS] 모델 세이브 경로: {model_save_path}")

    # 4-1) NLI로 학습된 모델 로드
    model = SentenceTransformer(nli_model_path)
    logging.info(f"[NLI->STS] Pre-trained NLI 모델 로드 완료: {nli_model_path}")

    # 4-2) KorSTS 데이터로 파인튜닝
    train_file_sts = os.path.join(KORSTS_PATH, "sts-train.tsv")
    dev_file_sts   = os.path.join(KORSTS_PATH, "sts-dev.tsv")

    train_samples_sts = load_kor_sts_samples(train_file_sts)
    dev_samples_sts   = load_kor_sts_samples(dev_file_sts)

    train_dataloader = DataLoader(train_samples_sts, shuffle=True, batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)

    dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples_sts, batch_size=batch_size, name='sts-dev')

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
    logging.info(f"NLI->STS warmup steps: {warmup_steps}")

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=dev_evaluator,
        epochs=num_epochs,
        evaluation_steps=1000,
        warmup_steps=warmup_steps,
        output_path=model_save_path
    )
    logging.info("[NLI->STS] 학습 완료!")
    return model_save_path

In [24]:
# -- 실험 4: NLI -> STS
nli_sts_path = train_nli_then_sts(
    nli_model_path=nli_model_path,
    max_seq_length=128,
    batch_size=16,
    num_epochs=4,
    output_dir=OUTPUT_DIR
)
final_scores['nli_then_sts'] = evaluate_on_sts_test(nli_sts_path, batch_size=8)
print(f"[NLI->STS] Test Score: {final_scores['nli_then_sts']}")
print(nli_sts_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.868666,0.868639
720,0.024500,No log,0.874131,0.875121
1000,0.011300,No log,0.878711,0.877995
1080,0.011300,No log,0.878163,0.877612
1440,0.011300,No log,0.878989,0.878044


[NLI->STS] Test Score: {'sts-test_pearson_cosine': 0.8492899426478029, 'sts-test_spearman_cosine': 0.8554136874109158}
output/nli-to-sts-2025-01-17


In [25]:
import shutil

# 압축할 폴더와 생성할 zip 파일 경로 설정
folder_to_zip = "/content/output"
zip_file_path = "/content/output.zip"

# 폴더를 압축
shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_to_zip)
print(f"폴더가 압축되었습니다: {zip_file_path}")

폴더가 압축되었습니다: /content/output.zip
